In [1]:
import sys
print('iPython Kernel -->', sys.argv)
print('Environment -->', sys.prefix)

iPython Kernel --> ['/Users/bravandi/anaconda3/envs/rbn/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/bravandi/Library/Jupyter/runtime/kernel-a8e4b66a-ea45-4509-89a7-c560b5d1d293.json']
Environment --> /Users/bravandi/anaconda3/envs/rbn


In [2]:
import pandas as pd
import numpy as np
import json
from collections import OrderedDict 

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

import random

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import seaborn as sns

import networkx as nx
# import GraphHierarchy as gh

from tqdm import tqdm

from scipy import stats

import gspread

'''
This layout algorithm requires installin graphviz
https://pygraphviz.github.io/documentation/stable/install.html
'''
from networkx.drawing.nx_agraph import graphviz_layout

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_colwidth', 0)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

%matplotlib inline

/var/folders/ng/_0t6l_610xd9vdp28t5_x7c80000gn/T/ipykernel_40746/1995286510.py:36: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
import os
from pandas import ExcelWriter


def save_xls(dfs_dict, xls_path, open=False):
    if type(dfs_dict) is not dict:
        dfs_dict = {'sheet1': dfs_dict}
    
    with ExcelWriter(xls_path) as writer:
        for df_name, df in dfs_dict.items():
            df.to_excel(writer, df_name)
        writer.save()

    if open is True:
        os.system('start EXCEL.EXE "{}"'.format(os.path.abspath(xls_path)))
    pass

# Load Networks

In [8]:
centuries = {
    '11': 11.0, '12': 12.0, '13': 13.0, '14E': 14.1, '14L': 14.2, '15': 15.0, '16': 16.0, '17': 17.0
}

## Check Mistake Duplicate Edges & Add Frequency

THe only exception of having multiple interactions between a source and target is in 17th century as below. ALl other centuries must have only a single interaction (type of edge) between two nodes.

Source:'PR ze'	 |	Target: 'IV werden'   |   Interactions: {PP, VP}

In [9]:
gc = gspread.service_account(filename='google_service_credentials.json')

In [10]:
google_sheet_ID = '1CWbPlXrKa41DL02mO6pny8BC4Hd7xwbIhA3-ZthyjWc'

german_dataset_gsheet = gc.open_by_key(google_sheet_ID)

german_dataset_gsheet.worksheets()

[<Worksheet '11' id:2063048793>,
 <Worksheet '12' id:936686375>,
 <Worksheet '13' id:698480864>,
 <Worksheet '14E' id:1758104757>,
 <Worksheet '14L' id:1538902914>,
 <Worksheet '15' id:2099895106>,
 <Worksheet '16' id:121866148>,
 <Worksheet '17' id:1360439213>,
 <Worksheet 'Descriptions' id:646251015>,
 <Worksheet 'Shortkeys!' id:1846877574>]

In [11]:
german_dataset_gsheet

<Spreadsheet 'Syntactic Netwroks German' id:1CWbPlXrKa41DL02mO6pny8BC4Hd7xwbIhA3-ZthyjWc>

In [25]:
cent_interaction_validate = {}
cent_raw_df = {}
cent_df_dict = {}
cent_all_df = []
'''This is to have the option of more complex network'''
enforce_single_function = True

for cent in centuries.keys():

    print('------------ Cent:', cent)
    
    cent_df = pd.DataFrame(german_dataset_gsheet.worksheet(cent).get_all_records())
    
    cent_df['cent'] = cent
    
    cent_df.loc[:, ['Source', 'Target', 'Interaction']] = (
        cent_df.loc[:, ['Source', 'Target', 'Interaction']].apply(lambda x: x.str.strip(), axis=1)
    ) 
    
    cent_df['Source Funcs All'] = cent_df['Source'].apply(lambda x: x.split(' ')[0])
    cent_df['Source Func Single'] = cent_df['Source Funcs All'].apply(lambda x: x.split('/')[0])

    cent_df['Target Funcs All'] = cent_df['Target'].apply(lambda x: x.split(' ')[0])
    cent_df['Target Func Single'] = cent_df['Target Funcs All'].apply(lambda x: x.split('/')[0])

    cent_df['Source Single Func'] = cent_df['Source Func Single'] + ' ' + cent_df['Source'].apply(lambda x: ' '.join(x.split(' ')[1:]))
    cent_df['Target Single Func'] = cent_df['Target Func Single'] + ' ' + cent_df['Target'].apply(lambda x: ' '.join(x.split(' ')[1:]))

    cent_df['Source All Funcs'] = cent_df['Source']
    cent_df['Target All Funcs'] = cent_df['Target']

    if enforce_single_function is True:
        cent_df['Source'] = cent_df['Source Single Func']
        cent_df['Target'] = cent_df['Target Single Func']

    cent_raw_df[cent] = cent_df.copy()
    
    """"""
    G=nx.from_pandas_edgelist(
        cent_df, source='Source', target='Target', 
        edge_attr=True, create_using=nx.DiGraph()
    )
    
    # nx.draw(G, with_labels=True)
    # print(type(G))
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)

    print('num raw edges in dataset:', len(cent_df), ' num network edges:', G.number_of_edges())
    """"""

    cent_df['frequency'] = cent_df['Interaction']

    cent_df = (
        cent_df.groupby(['Source', 'Target'])
        .agg({
            'Interaction': lambda x: set([v.upper() for v in x]),
            'frequency': np.size,
         })
        .reset_index()
    )

    cent_df['Interaction count'] = cent_df['Interaction'].apply(len)

    # print(path_net)
    cent_df = cent_df.sort_values(by=['Interaction count', 'frequency'], ascending=False)

    cent_df_dict[cent] = cent_df
    
    cent_interaction_validate[cent] = cent_df[cent_df['Interaction count'] > 1]  
    
    cent_all_df.append(cent_raw_df[cent])
    
    print(
        'Interaction count > 1 -->', len(cent_interaction_validate[cent])
    )
    
    del cent_df
#     break
    pass
# cent_df_dict.keys()
# cent_df = cent_raw_df[cent]
cent_all_df = pd.concat(cent_all_df).reset_index(drop=True)

print('IMP enforce_single_function:', enforce_single_function)

------------ Cent: 11
num raw edges in dataset: 968  num network edges: 725
Interaction count > 1 --> 0
------------ Cent: 12
num raw edges in dataset: 1194  num network edges: 803
Interaction count > 1 --> 0
------------ Cent: 13
num raw edges in dataset: 2503  num network edges: 1330
Interaction count > 1 --> 0
------------ Cent: 14E
num raw edges in dataset: 2326  num network edges: 1321
Interaction count > 1 --> 0
------------ Cent: 14L
num raw edges in dataset: 2494  num network edges: 1586
Interaction count > 1 --> 0
------------ Cent: 15
num raw edges in dataset: 2227  num network edges: 1424
Interaction count > 1 --> 0
------------ Cent: 16
num raw edges in dataset: 3346  num network edges: 2448
Interaction count > 1 --> 0
------------ Cent: 17
num raw edges in dataset: 3940  num network edges: 3083
Interaction count > 1 --> 0
IMP enforce_single_function: True


In [26]:
print('WE MUST HAVE Only two nodes with multiple edges!')
print('NOT ANYMORE! we fixed it')
cent_interaction_validate['17']

WE MUST HAVE Only two nodes with multiple edges!
NOT ANYMORE! we fixed it


,Source,Target,Interaction,frequency,Interaction count


In [27]:
cent_all_df['Communicative Need'].value_counts()

                                          14682
Passive Voice                             2504 
Change of State                           973  
Future Reference                          398  
Obligation/Permission/Future Reference    161  
Obligation                                97   
Capacities                                95   
Desires/Wants                             46   
Obligation/Permission                     18   
Capacities/Desires                        9    
Desires                                   8    
Permission                                3    
-                                         2    
Being Powerful                            1    
?                                         1    
Name: Communicative Need, dtype: int64